In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, r2_score

file_path = './datasets/turkey_cars.csv'
cars_data = pd.read_csv(file_path, encoding='ISO-8859-9')
cars_data.columns = cars_data.columns.str.replace('?', 'i')

# Separate the target variable (y) from the predictors (X)
cars_data_with_color = cars_data.copy().drop(['Fiyat'], axis=1)
X = cars_data.drop(columns=['Fiyat', 'ilan Tarihi', 'Renk'], axis=1)
y = cars_data['Fiyat']

print(X.columns.values)
print(y.values)

In [ ]:

# Selecting categorical columns with relatively low cardinality for one-hot encoding
categorical_cols = [cname for cname in X.columns if X[cname].nunique() < 100 and 
                    X[cname].dtype == "object" and cname not in ['ilan Tarihi', 'Renk']]

# Selecting numerical columns
numerical_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]

# Keeping selected columns only
my_cols = categorical_cols + numerical_cols
X = X[my_cols]

print(categorical_cols)
print(numerical_cols)
print(X.columns.values)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(X_train.shape)
print(X_test.shape)

print(X_train[0:5]['Km'])

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler 
from sklearn.compose import ColumnTransformer

# Preprocessing for numerical data
numerical_transformer = StandardScaler()  # Data standardization with standard deviation and mean

# Preprocessing for categorical data
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

print(X_train_transformed.shape)
print(X_train_transformed[0:5][0][0])

In [ ]:
correlation_matrix = cars_data[numerical_cols + ['Fiyat']].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
# Calculate the average price for each brand
avg_price_per_brand = cars_data.groupby('Marka')['Fiyat'].mean()

# Sort the results in descending order
avg_price_per_brand = avg_price_per_brand.sort_values(ascending=False)

# Plot the results
plt.figure(figsize=(10, 10))
avg_price_per_brand.plot(kind='barh')
plt.title('Average Price per Brand')
plt.xlabel('Average Price')
plt.ylabel('Brand')
plt.gca().invert_yaxis()  # invert the y-axis to have the highest value at the top
plt.show()

In [ ]:
# Calculate the average price for each color
avg_price_per_color = cars_data.groupby('Renk')['Fiyat'].mean()

# Sort the results in descending order
avg_price_per_color = avg_price_per_color.sort_values(ascending=False)

# Plot the results
plt.figure(figsize=(10, 10))
avg_price_per_color.plot(kind='barh')
plt.title('Average Price per Color')
plt.xlabel('Average Price')
plt.ylabel('Color')
plt.gca().invert_yaxis()  # invert the y-axis to have the highest value at the top
plt.show()

In [ ]:
# Count the occurrences of each value in the 'vites' column
vites_counts = cars_data['Vites'].value_counts()

# Plot the results
plt.figure(figsize=(10, 6))
vites_counts.plot(kind='pie', autopct='%1.1f%%')
plt.title('Distribution of Vites')
plt.ylabel('')  # Hide the y-axis label
plt.show()

In [ ]:
# RANDOM FOREST REGRESSION
from sklearn.ensemble import RandomForestRegressor

# Define the model
rf = RandomForestRegressor(n_estimators=100, random_state=0)

# Preprocessing of training data, fit model 
rf.fit(X_train_transformed, y_train)

# Preprocessing of validation data, get predictions
y_pred_rf = rf.predict(X_test_transformed)

print('R2 score for RF:', r2_score(y_test, y_pred_rf))
print('MAE for RF:', mean_absolute_error(y_test, y_pred_rf))
print('MAPE for RF:', mean_absolute_percentage_error(y_test, y_pred_rf) * 100, '%')

In [ ]:
# Extracting feature importances from the model within the pipeline
feature_importances = rf.feature_importances_

# Getting the list of original feature names after one-hot encoding
encoded_features = preprocessor.transformers_[1][1].get_feature_names_out(categorical_cols)
feature_names = numerical_cols + list(encoded_features)

# Mapping these importances back to the original feature names 
# (including aggregating importances for categories of each original categorical feature)
importance_dict = dict(zip(feature_names, feature_importances))
original_feature_importances = {feature: 0 for feature in my_cols}

for feature, importance in importance_dict.items():
    if feature in original_feature_importances:
        original_feature_importances[feature] += importance
    else:
        original_feature_name = feature.split('_')[0]  # Extracting the original feature name before encoding
        original_feature_importances[original_feature_name] += importance

# Sorting the features by importance
sorted_feature_importances = sorted(original_feature_importances.items(), key=lambda x: x[1], reverse=True)

# Converting to DataFrame for easier plotting
df_feature_importances = pd.DataFrame(sorted_feature_importances, columns=['Feature', 'Importance'])

# Plotting
plt.figure(figsize=(12, 6))
sns.barplot(x='Importance', y='Feature', data=df_feature_importances, palette="rocket", hue='Feature')
plt.title('Feature Importance for Predicting Car Price')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

In [ ]:
# SUPPORT VECTOR REGRESSION
from sklearn.svm import SVR

svr = SVR(kernel='linear')
svr.fit(X_train_transformed, y_train)
y_pred_svr = svr.predict(X_test_transformed)

print('R2 score for SVR:', r2_score(y_test, y_pred_svr))
print(f"MAE for SVR: {mean_absolute_error(y_test, y_pred_svr)}")
print('MAPE for SVR:', mean_absolute_percentage_error(y_test, y_pred_svr) * 100, '%')

In [ ]:
# LINEAR and POLYNOMIAL REGRESSION
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

pr = PolynomialFeatures(degree=1)
X_poly = pr.fit_transform(X_train_transformed)

lr = LinearRegression()
lr.fit(X_poly, y_train)
X_poly_test = pr.fit_transform(X_test_transformed)

y_pred_poly = lr.predict(X_poly_test)

print(f"R2 score for POLY: {r2_score(y_test, y_pred_poly)}")
print(f"MAE for POLY: {mean_absolute_error(y_test, y_pred_poly)}")
print('MAPE for SVR:', mean_absolute_percentage_error(y_test, y_pred_poly) * 100, '%')

In [ ]:
# LINEAR Line with Km vs Fiyat
X_km = cars_data[['Km']].values  # Predictor
y_fiyat = cars_data['Fiyat'].values  # Response

# Fit the linear regression model
model = LinearRegression()
model.fit(X_km, y_fiyat)

# Generate a range of values for Km to create a prediction line
X_km_range = np.linspace(X_km.min(), X_km.max(), 100).reshape(-1, 1)
y_fiyat_pred = model.predict(X_km_range)

# Plotting
plt.figure(figsize=(10, 6))
plt.scatter(X_km, y_fiyat, color='blue', label='Actual data', alpha=0.5)
plt.plot(X_km_range, y_fiyat_pred, color='red', label='Linear regression line')
plt.title('Km vs Fiyat with Linear Regression')
plt.xlabel('Km')
plt.ylabel('Fiyat')
plt.legend()
plt.show()

In [ ]:
import numpy as np
from sklearn.inspection import permutation_importance

# Fit your model
lr.fit(X_train_transformed, y_train)

X_test_dense = X_test_transformed.toarray()
print(X_test_dense.shape)
feature_names = preprocessor.get_feature_names_out(input_features=X_train.columns)

# Create a PermutationImportance object
results = permutation_importance(lr, X_test_dense, y_test, scoring='neg_mean_squared_error')

# Get importance
importance = results.importances_mean

# Create a list of (feature, importance) tuples
importances = list(zip(feature_names, importance))

# Filter out features with nan importance
importances = [(feature, score) for feature, score in importances if not np.isnan(score)]

# Calculate the total importance
total_importance = sum(abs(score) for feature, score in importances)

# Calculate weights and create a list of (feature, weight, importance) tuples
importances = [(feature, abs(score) / total_importance, score) for feature, score in importances]

# Sort the importances by weight in descending order
importances.sort(key=lambda x: x[1], reverse=True)

# Summarize feature importance
for feature, weight, importance in importances:
    print('Feature: %s, Weight: %.5f, Score: %.5f' % (feature.split('__')[1], weight, importance))

In [ ]:
import matplotlib.pyplot as plt

# Get the top 10 and bottom 10 features by weight
top_10 = importances[:10]
bottom_10 = importances[-10:]

# Create a new figure with 2 subplots side by side
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Plot the top 10 features
axs[0].barh([x[0].split('__')[1] for x in top_10], [x[1] for x in top_10])
axs[0].set_title('Top 10 Features')
axs[0].invert_yaxis()

# Plot the bottom 10 features
axs[1].barh([x[0].split('__')[1] for x in bottom_10], [x[1] for x in bottom_10])
axs[1].set_title('Bottom 10 Features')
axs[1].invert_yaxis()

# Display the plot
plt.tight_layout()
plt.show()

In [ ]:
# RIDGE REGRESSION
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=1)
ridge.fit(X_train_transformed, y_train)
y_pred_ridge = ridge.predict(X_test_transformed)

print(f"R2 score for Ridge: {r2_score(y_test, y_pred_ridge)}")
print(f"MAE for Ridge: {mean_absolute_error(y_test, y_pred_ridge)}")
print('MAPE for Ridge:', mean_absolute_percentage_error(y_test, y_pred_ridge) * 100, '%')

In [ ]:
# NEURAL NETWORK REGRESSION
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor(hidden_layer_sizes=(8,16,32,16), activation='relu', solver='adam', max_iter=10000, random_state=42)
mlp.fit(X_train_transformed, y_train)
y_pred_mlp = mlp.predict(X_test_transformed)

print(f"R2 score for MLP: {r2_score(y_test, y_pred_mlp)}")
print(f"MAE for MLP: {mean_absolute_error(y_test, y_pred_mlp)}")
print('MAPE for MLP:', mean_absolute_percentage_error(y_test, y_pred_mlp) * 100, '%')

In [ ]:
import pandas as pd

print(cars_data['Vites'].unique())

dummy_data = {
    'ilan Tarihi': '01/01/2020',
    'Marka': 'Toyota',
    'Arac Tip Grubu': 'Sedan',
    'Arac Tip': '2.0 D-4D',
    'Model Yil': 2015,
    'Yakit Turu': 'Dizel',
    'Vites': 'Yar? Otomatik Vites',
    'CCM': '1601-1800 cc',
    'Beygir Gucu': '101-125 BG',
    'Renk': 'Siyah',
    'Kasa Tipi': 'Hatchback 5 Kapı',
    'Kimden': 'Sahibinden',
    'Durum': 'İkinci El',
    'Km': 60000,
}

dummy_data_df = pd.DataFrame([dummy_data])  # Convert to DataFrame
dummy_X = preprocessor.transform(dummy_data_df)

prediction = rf.predict(dummy_X)

print(prediction)